In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [3]:
# def build_data():
    
#     import re
    
#     p = re.compile(r'(.+?)/(?:([a-z]{1,2})(?:$| ))')
    
#     raw_data_dir = os.path.dirname(data_path)
#     seg_file = os.path.join(raw_data_dir, 'final_baidu-23w.txt')
    
#     fake_file = os.path.join(raw_data_dir, 'fake.txt')
    
#     delimiter='△△△'
    
#     replace_chars = [
#         '\x97',
#         '\uf076',
#         "\ue405",
#         "\ue105",
#         "\ue415",
#         '\x07',
#         '\x7f',
#         '\u3000',
#         '\xa0',
#         ' '
#     ]
#     with open(seg_file) as fin1, \
#           open(fake_file) as fin2, \
#           open(train_path, 'w') as train_f, \
#           open(valid_path, 'w') as valid_f:
        
#         train_f.write(f'0{delimiter}1\n')
#         valid_f.write(f'0{delimiter}1\n')
        
#         fins = [fin1, fin2]
#         for fin in fins:
#             for line in fin:
#                 line = line.strip()
#                 if not line:
#                     continue

#                 import random
#                 score = random.random()

#                 fout = train_f if score > 0.006 else valid_f
#                 words = []
#                 flags = []
#                 for word, flag in p.findall(line):
#                     char_list = ['unk' if c in replace_chars or c.isspace() else c for c in list(word)]

#                     char_size = len(char_list)
#                     if char_size == 1:
#                         # 一些错误的单个字符实体剔除掉
#                         if flag in ['nt', 'ti', 'nr', 'ns', 'nz']:
#                             flag = 'xx'
#                         # 单个
#                         tag_list = [f'S_{flag}']
#                     else:
#                         tag_list = [f'B_{flag}'] + [f'I_{flag}']  * (len(char_list) - 2) + [f'E_{flag}']

#                     if char_size != len(tag_list):
#                         print(line)
#                         print(word, flag)
#                         print(char_list, tag_list)

#                     words.extend(char_list)
#                     flags.extend(tag_list)

#                 assert len(words) == len(flags)

#                 fout.write(delimiter.join([
#                     ' '.join(flags),
#                     ' '.join(words)
#                 ]))
#                 fout.write('\n')
            
            
            
# build_data()
    

In [ ]:
# 正常训练

from bailian_nlp.modules import BertNerData as NerData

data = NerData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    is_cls=False,
    max_seq_len=64,
    batch_size=32
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


2019-04-13 15:20:04,506 DEBUG: get_data cost 177.531658s


2019-04-13 15:20:05,502 DEBUG: get_data cost 0.890404s


2019-04-13 15:20:10,737 INFO: Resuming train... Current epoch 0.


In [ ]:
num_epochs = 2
learner.fit(num_epochs, target_metric='f1')

In [9]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

learner = tagger.learner
num_epochs = 5
learner.load_model()
# learner.fit(num_epochs, target_metric='f1')


2019-04-13 13:52:57,351 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-13 13:52:57,611 INFO: pos model loads success!


2019-04-13 13:54:28,405 DEBUG: get_data cost 89.178048s


2019-04-13 13:54:28,986 DEBUG: get_data cost 0.579848s


2019-04-13 13:54:29,794 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-13 13:54:30,055 INFO: pos model loads success!


In [19]:
type(learner.data.train_dl)

modules.data.bert_data.DataLoaderForTrain

In [ ]:
learner.save_model()

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [7]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


2019-04-13 15:01:38,399 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-13 15:01:38,714 INFO: pos model loads success!


In [17]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res

2019-04-13 15:14:04,769 DEBUG: get_data cost 0.036086s


2019-04-13 15:14:04,873 DEBUG: text_array_for_predict cost 0.141047s



0.14226937294006348


[[('周光', 'nr'),
  ('明确', 'ad'),
  ('否认', 'v'),
  ('CEO', 'ti'),
  ('佟显侨', 'nr'),
  ('和', 'c'),
  ('CTO', 'ti'),
  ('衡量', 'v'),
  ('说', 'v'),
  ('的', 'u'),
  ('罪名', 'n')]]